In [1]:
import pandas as pd
import numpy as np
import re
# Set display options to show full content in columns
pd.set_option('display.max_colwidth', None)
import sys
from pathlib import Path

# Get the path to the current script
current_dir = Path.cwd()

# Go one level up
current_dir = current_dir.parent

# Add the 'scripts' directory to sys.path to be able to import data_utils.py
sys.path.append(str(current_dir))

from scripts.data_utils import split_summary_methods
import ast  
import matplotlib.pyplot as plt
import seaborn as sns
from autorank import autorank, plot_stats, create_report

c:\Users\nicol\spectral_anomaly_detector\scripts\data_utils.py:2: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('mode.use_inf_as_na', True)


In [2]:
summary_results = pd.read_excel(current_dir / "datasets" / "summaries" / "summary_results_ablation_study_pv_ds.xlsx")
summary_results.method_window_and_param.unique()

array(["BKNN_60_{'algorithm': 'brute', 'chunk_size': 1, 'dmetric': 'cityblock', 'ensemble_size': 50, 'n_jobs': -1, 'no_bootstrapp': False, 'no_z_score': True, 'transf': 'NONE'}",
       "BKNN_60_{'algorithm': 'brute', 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'no_bootstrapp': False, 'no_z_score': True, 'transf': 'NONE'}",
       "BKNN_60_{'algorithm': 'brute', 'chunk_size': 10, 'dmetric': 'cityblock', 'ensemble_size': 50, 'n_jobs': -1, 'no_bootstrapp': False, 'no_z_score': True, 'transf': 'NONE'}",
       "BKNN_60_{'algorithm': 'brute', 'chunk_size': 1, 'dmetric': 'cityblock', 'ensemble_size': 10, 'n_jobs': -1, 'no_bootstrapp': False, 'no_z_score': True, 'transf': 'NONE'}",
       "BKNN_60_{'algorithm': 'brute', 'chunk_size': 10, 'dmetric': 'cityblock', 'ensemble_size': 1, 'n_jobs': -1, 'no_bootstrapp': False, 'no_z_score': True, 'transf': 'NONE'}",
       "BKNN_60_{'algorithm': 'brute', 'chunk_size': 50, 'dmetric': 'cityblock', 'ensemble_size': 240

In [3]:


# Sample pivot table (replace this with your pivot table)
duplicates = summary_results[["iteration", "scenario", "method_window_and_param", "count_cleaned_score"]].duplicated(keep=False)
summary_results[duplicates][["iteration", "scenario", "method_window_and_param", "count_cleaned_score"]].sort_values(by=["method_window_and_param"])



,iteration,scenario,method_window_and_param,count_cleaned_score


In [4]:


pivot = summary_results.pivot_table(values=["count_cleaned_score", "count_anomalies"], 
                                  index=["scenario"], aggfunc='mean')

# Calculate the percentage of anomalies
pivot['percentage_anomalies'] = (pivot['count_anomalies'] / pivot['count_cleaned_score']) * 100

# Display the sorted pivot table
pivot



,count_anomalies,count_cleaned_score,percentage_anomalies
scenario,,,
A4,60.0,4200.0,1.428571
A5,300.0,4200.0,7.142857
A6,600.0,4200.0,14.285714


In [5]:
summary_results[['method',"window_size", 'parameters']] = summary_results['method_window_and_param'].apply(
    lambda x: pd.Series(split_summary_methods(x))
)
# Sample pivot table (replace this with your pivot table)
pivot = summary_results.pivot_table(values=["count_cleaned_score"], 
                                  index=['method',"window_size", 'parameters', "iteration", "scenario"], aggfunc='sum').reset_index()

pivot = pivot[pivot.count_cleaned_score != 4200]
pivot["window_and_iter_scenario"] = "Window: "+pivot["window_size"].astype(str) + " - Iteration: " + pivot["iteration"].astype(str) + " - Scenario: " + pivot["scenario"].astype(str) +" - Total: " + pivot["count_cleaned_score"].astype(str)
# Display the sorted pivot table
print(pivot.window_and_iter_scenario.unique())


[]


In [6]:
summary_results.columns

Index(['iteration', 'scenario', 'method_window_and_param', 'AUC_ROC', 'AUC_PR',
       'Max_F1', 'mean_training_time', 'max_training_time',
       'min_training_time', 'mean_scoring_time', 'max_scoring_time',
       'min_scoring_time', 'count_cleaned_score', 'count_raw_score',
       'count_anomalies', 'count_normal', 'method', 'window_size',
       'parameters'],
      dtype='object')

In [7]:
# Apply function to create new columns
summary_results[['method',"window_size", 'parameters']] = summary_results['method_window_and_param'].apply(
    lambda x: pd.Series(split_summary_methods(x))
)
summary_results["params_with_window"] = summary_results["parameters"]+ "_{'window_size':" + summary_results['window_size']+"}"
summary_results["params_with_window"] = summary_results["params_with_window"].str.replace("}_{", ",", regex=False)


regex_pattern = r"'k':\s*(\d+)"

# Use .str.extract() to pull out the captured digits
summary_results['k_parameter'] = summary_results['params_with_window'].str.extract(regex_pattern).replace(np.nan, "1")


# Define the order you want for the 'scenario' column
order = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

# Convert the 'scenario' column to a categorical type with the specified order
summary_results['scenario'] = pd.Categorical(summary_results['scenario'], categories=order, ordered=True)



In [8]:
contain_regex_filter = "|".join(["^(?=.*OnlineBootKNN_)(?=.*cityblock).*$","SWKNN_"])

no_contain_regex_filter = "|".join(["^(?=.*OnlineBootKNN_)(?=.*SQRT).*$"])

filtered_data = summary_results[summary_results['method_window_and_param'].str.contains(contain_regex_filter, regex=True)]

filtered_data = filtered_data[~filtered_data['method_window_and_param'].str.contains(no_contain_regex_filter, regex=True)]


#filtered_data = filtered_data[filtered_data['method'].str.contains("std_p")]
#filtered_data = filtered_data[filtered_data['method'].str.contains("None|FOD|SOD|DIL|QUANT")]
#filtered_data = filtered_data[filtered_data['method'].str.contains("QUANT")]
#filtered_data = filtered_data[filtered_data.method.str.contains("OnlineBootGP|OnlineBootKNN")]

# Now, sort the data based on the 'scenario' column
filtered_data = filtered_data.sort_values(by='scenario')

#filtered_data = filtered_data[~filtered_data.window_size.isin(["60"])]

#filtered_data = filtered_data[filtered_data.method.isin(["ExactStorm", "OnlineBootKNN"])]
#filtered_data = filtered_data[filtered_data.scenario.isin(["A1", "A4","A7"])]




# average results over all iterations and all scenarios
averaged_results = filtered_data.groupby(['method', 'scenario', 'params_with_window',"k_parameter"])['AUC_PR'].agg(
    mean_auc_pr='mean',
    std_auc_pr='std',
    n_runs='count'
).reset_index()



C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\2296251610.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  averaged_results = filtered_data.groupby(['method', 'scenario', 'params_with_window',"k_parameter"])['AUC_PR'].agg(


In [9]:
total_scenarios = len(filtered_data.scenario.unique())

In [10]:
# Compute overall mean, std, and count across all datasets

summary = averaged_results.groupby(['method', 'params_with_window','k_parameter']).agg(
    mean_auc_pr=('mean_auc_pr', 'mean'),
    std_auc_pr=('std_auc_pr', 'mean'),
    std_mean_auc_pr=('mean_auc_pr', 'std'),
    n_scenarios=('mean_auc_pr', 'count')
).reset_index()


In [11]:
# Exclude the rows where the number of scenarios is not equal to total_scenarios
summary = summary[summary.n_scenarios==total_scenarios]

summary.sort_values(['mean_auc_pr','std_auc_pr','std_mean_auc_pr'], ascending=[False, True, True])

,method,params_with_window,k_parameter,mean_auc_pr,std_auc_pr,std_mean_auc_pr,n_scenarios
294,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}",1,0.933377,0.000272,0.008894,3
303,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 50, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}",1,0.933274,0.000234,0.008793,3
285,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 10, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}",1,0.933150,0.000013,0.008708,3
276,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 1, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}",1,0.931531,0.001675,0.007112,3
288,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':120}",1,0.929011,0.000054,0.003110,3
...,...,...,...,...,...,...,...
757,SWKNN,"{'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':120}",10,0.745028,0.000000,0.142297,3
763,SWKNN,"{'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}",10,0.706163,0.000000,0.207844,3
770,SWKNN,"{'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}",50,0.516891,0.000000,0.168540,3
767,SWKNN,"{'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':120}",50,0.516474,0.000000,0.168505,3


In [12]:
summary.columns

Index(['method', 'params_with_window', 'k_parameter', 'mean_auc_pr',
       'std_auc_pr', 'std_mean_auc_pr', 'n_scenarios'],
      dtype='object')

In [13]:
# Select the highest AUC_PR per method (after averaging across iterations)
best_results = (
    summary
    .sort_values(['mean_auc_pr','std_auc_pr','std_mean_auc_pr'], ascending=[False, True, True])
    .groupby(['method','k_parameter'])
    .head(1)
)

best_results


,method,params_with_window,k_parameter,mean_auc_pr,std_auc_pr,std_mean_auc_pr,n_scenarios
294,OnlineBootKNN,"{'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}",1,0.933377,0.000272,0.008894,3
753,SWKNN,"{'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}",1,0.918193,0.000000,0.064484,3
760,SWKNN,"{'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}",10,0.746040,0.000000,0.140554,3
770,SWKNN,"{'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}",50,0.516891,0.000000,0.168540,3


In [14]:
# Part 1: Select the columns you want to keep as they are
unformatted_cols = best_results[["method", "k_parameter", "mean_auc_pr"]].copy()
unformatted_cols["mean_auc_pr"] = unformatted_cols["mean_auc_pr"].round(3)
# Part 2: Select and format the other columns
formatted_cols = best_results[["std_mean_auc_pr", "std_auc_pr"]].applymap(lambda x: f"{x:.1e}")

# Combine them side-by-side for printing
pd.concat([unformatted_cols, formatted_cols], axis=1)

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\2243943095.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_cols = best_results[["std_mean_auc_pr", "std_auc_pr"]].applymap(lambda x: f"{x:.1e}")


,method,k_parameter,mean_auc_pr,std_mean_auc_pr,std_auc_pr
294,OnlineBootKNN,1,0.933,8.9e-03,2.7e-04
753,SWKNN,1,0.918,6.4e-02,0.0e+00
760,SWKNN,10,0.746,1.4e-01,0.0e+00
770,SWKNN,50,0.517,1.7e-01,0.0e+00


In [15]:
best_params = best_results['params_with_window'].unique().tolist()

# Ensure parameters are strings for consistent matching

# Filter rows where 'parameters' column contains any of the best_params
filtered_data = filtered_data[filtered_data['params_with_window'].isin(best_params)]



In [16]:
# Sample pivot table (replace this with your pivot table)
pivot = filtered_data.pivot_table(values=["count_cleaned_score", "count_anomalies"], 
                                  index=["scenario"], aggfunc='mean')

# Calculate the percentage of anomalies
pivot['percentage_anomalies'] = (pivot['count_anomalies'] / pivot['count_cleaned_score']) * 100

# Display the sorted pivot table
pivot

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3223993330.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = filtered_data.pivot_table(values=["count_cleaned_score", "count_anomalies"],


,count_anomalies,count_cleaned_score,percentage_anomalies
scenario,,,
A4,60.0,4200.0,1.428571
A5,300.0,4200.0,7.142857
A6,600.0,4200.0,14.285714


## Summary Score of Online Anomaly Detectors with PV Datasets 

In [17]:
# Pivot for mean
mean_pivot = filtered_data.pivot_table(
    values="AUC_PR",
    columns="scenario",
    index=["method", "params_with_window", "k_parameter"],
    aggfunc="mean"
)

# Pivot for std
std_pivot = filtered_data.pivot_table(
    values="AUC_PR",
    columns="scenario",
    index=["method", "params_with_window", "k_parameter"],
    aggfunc="std"
)

# Combine as "mean ± std"
combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.applymap(lambda x: f"{x:.1e}").astype(str)

# Calculate and append row-wise average of means
avg_col = mean_pivot.mean(axis=1).round(3)
combined["Avg"] = avg_col.astype(str) + " ± " + std_pivot.mean(axis=1).apply(lambda x: f"{x:.1e}").astype(str)

# Optional: sort by average
combined = combined.sort_values(by="Avg", ascending=False)

# Display with HTML formatting in Jupyter
combined


C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3537622017.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mean_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3537622017.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  std_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3537622017.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.applymap(lambda x: f"{x:.1e}").astype(str)


scenario                                                                                                                                                                                         A4  \
method        params_with_window                                                                                                                                       k_parameter                    
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60} 1            0.923 ± 0.0e+00   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                        1            0.844 ± 0.0e+00   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      10            0.59 ± 0.0e+00   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      50           0.338 ± 0.0e+00   

scenario                                                                                                                                                                                         A5  \
method        params_with_window                                                                                                                                       k_parameter                    
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60} 1             0.94 ± 4.0e-04   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                        1             0.95 ± 0.0e+00   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      10           0.784 ± 0.0e+00   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      50            0.54 ± 0.0e+00   

scenario                                                                                                                                                                                         A6  \
method        params_with_window                                                                                                                                       k_parameter                    
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60} 1            0.937 ± 4.1e-04   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                        1             0.96 ± 0.0e+00   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      10           0.864 ± 0.0e+00   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                      50           0.673 ± 0.0e+00   

scenario                                                                                                                                                                                        Avg  
method        params_with_window                                                                                                                                       k_parameter                   
OnlineBootKNN {'alg

In [18]:
"""
mean_pivot = filtered_data.pivot_table(
    values="AUC_PR",
    index="scenario",
    columns=["method","params_with_window"],
    aggfunc="mean"
)

result = autorank(mean_pivot, alpha=0.05, verbose=True, order='descending')
#print(result)

create_report(result)

plot_stats(result, allow_insignificant=True)
plt.show()
"""


'\nmean_pivot = filtered_data.pivot_table(\n    values="AUC_PR",\n    index="scenario",\n    columns=["method","params_with_window"],\n    aggfunc="mean"\n)\n\nresult = autorank(mean_pivot, alpha=0.05, verbose=True, order=\'descending\')\n#print(result)\n\ncreate_report(result)\n\nplot_stats(result, allow_insignificant=True)\nplt.show()\n'

In [19]:
# Pivot for mean
mean_pivot = filtered_data.pivot_table(
    values="AUC_ROC",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="mean"
)

# Pivot for std
std_pivot = filtered_data.pivot_table(
    values="AUC_ROC",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="std"
)

# Combine as "mean ± std"
combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.round(3).astype(str)

# Calculate and append row-wise average of means
avg_col = mean_pivot.mean(axis=1).round(3)
combined["Avg"] = avg_col.astype(str) + " ± " + std_pivot.mean(axis=1).round(3).astype(str)

# Optional: sort by average
# combined = combined.sort_values(by="Avg", ascending=False)

# Display with HTML formatting in Jupyter
combined

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\1541842640.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mean_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\1541842640.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  std_pivot = filtered_data.pivot_table(


scenario                                                                                                                                                                         A4  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}    1.0 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.983 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.996 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.988 ± 0.0   

scenario                                                                                                                                                                         A5  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.997 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.993 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.993 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.975 ± 0.0   

scenario                                                                                                                                                                         A6  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.992 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.995 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.993 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.972 ± 0.0   

scenario                                                                                                                                                                        Avg  
method        params_with_window                                                                                                                                                     
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.996 ± 0.0  
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}              

In [20]:
# Pivot for mean
mean_pivot = filtered_data.pivot_table(
    values="Max_F1",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="mean"
)

# Pivot for std
std_pivot = filtered_data.pivot_table(
    values="Max_F1",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="std"
)

# Combine as "mean ± std"
combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.round(3).astype(str)

# Calculate and append row-wise average of means
avg_col = mean_pivot.mean(axis=1).round(3)
combined["Avg"] = avg_col.astype(str) + " ± " + std_pivot.mean(axis=1).round(3).astype(str)

# Optional: sort by average
# combined = combined.sort_values(by="Avg", ascending=False)

# Display with HTML formatting in Jupyter
combined

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\928999621.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mean_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\928999621.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  std_pivot = filtered_data.pivot_table(


scenario                                                                                                                                                                         A4  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.976 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.967 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.863 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.682 ± 0.0   

scenario                                                                                                                                                                         A5  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.943 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.992 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.958 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.851 ± 0.0   

scenario                                                                                                                                                                         A6  \
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.949 ± 0.0   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                         0.994 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.978 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                       0.919 ± 0.0   

scenario                                                                                                                                                                        Avg  
method        params_with_window                                                                                                                                                     
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.956 ± 0.0  
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}              

## Summary Training Time of Online Anomaly Detectors with PV Datasets 

In [21]:
# Pivot for mean
mean_pivot = filtered_data.pivot_table(
    values="mean_training_time",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="mean"
)

# Pivot for std
std_pivot = filtered_data.pivot_table(
    values="mean_training_time",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="std"
)

# Combine as "mean ± std"
combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.round(3).astype(str)

# Calculate and append row-wise average of means
avg_col = mean_pivot.mean(axis=1).round(3)
combined["Avg"] = avg_col.astype(str) + " ± " + std_pivot.mean(axis=1).round(3).astype(str)

# Optional: sort by average
# combined = combined.sort_values(by="Avg", ascending=False)

# Display with HTML formatting in Jupyter
combined

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3217713745.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mean_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3217713745.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  std_pivot = filtered_data.pivot_table(


scenario                                                                                                                                                                           A4  \
method        params_with_window                                                                                                                                                        
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.299 ± 0.001   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                             0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   

scenario                                                                                                                                                                           A5  \
method        params_with_window                                                                                                                                                        
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.312 ± 0.001   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                             0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   

scenario                                                                                                                                                                           A6  \
method        params_with_window                                                                                                                                                        
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.108 ± 0.004   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                             0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                           0.0 ± 0.0   

scenario                                                                                                                                                                         Avg  
method        params_with_window                                                                                                                                                      
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.24 ± 0.002  
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampl

## Summary Scoring Time of Online Anomaly Detectors with PV Datasets 

In [22]:
# Pivot for mean
mean_pivot = filtered_data.pivot_table(
    values="mean_scoring_time",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="mean"
)

# Pivot for std
std_pivot = filtered_data.pivot_table(
    values="mean_scoring_time",
    columns="scenario",
    index=["method", "params_with_window"],
    aggfunc="std"
)

# Combine as "mean ± std"
combined = mean_pivot.round(3).astype(str) + " ± " + std_pivot.round(3).astype(str)

# Calculate and append row-wise average of means
avg_col = mean_pivot.mean(axis=1).round(3)
combined["Avg"] = avg_col.astype(str) + " ± " + std_pivot.mean(axis=1).round(3).astype(str)

# Optional: sort by average
# combined = combined.sort_values(by="Avg", ascending=False)

# Display with HTML formatting in Jupyter
combined

C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3233268288.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  mean_pivot = filtered_data.pivot_table(
C:\Users\nicol\AppData\Local\Temp\ipykernel_4000\3233268288.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  std_pivot = filtered_data.pivot_table(


scenario                                                                                                                                                                           A4  \
method        params_with_window                                                                                                                                                        
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.426 ± 0.002   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                             0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                         0.001 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                         0.001 ± 0.0   

scenario                                                                                                                                                                           A5  \
method        params_with_window                                                                                                                                                        
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.424 ± 0.001   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                             0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                         0.001 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                         0.001 ± 0.0   

scenario                                                                                                                                                                          A6  \
method        params_with_window                                                                                                                                                       
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.27 ± 0.001   
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':60}                            0.0 ± 0.0   
              {'k': 10, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                        0.001 ± 0.0   
              {'k': 50, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling': 5,'window_size':240}                        0.001 ± 0.0   

scenario                                                                                                                                                                          Avg  
method        params_with_window                                                                                                                                                       
OnlineBootKNN {'algorithm': 'brute', 'alpha': 0.05, 'chunk_size': 240, 'dmetric': 'cityblock', 'ensemble_size': 240, 'n_jobs': -1, 'transf': 'ZNORM','window_size':60}  0.373 ± 0.001  
SWKNN         {'k': 1, 'k_is_max': False, 'max_node_size': 20, 'metric': 'cityblock', 'min_node_size': 5, 'split_sampling